### Настройка Airflow

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься настройкой среды исполнения, а сразу начать писать код и работать с Airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4
!pip install wtforms==2.3.3
!pip install airflow.providers.http

# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags

In [41]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.

In [42]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.

In [51]:
# Запуск шедулера
!airflow scheduler -D

/usr/local/lib/python3.10/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'Serialize

In [44]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken  # найти его можно https://dashboard.ngrok.com/get-started/setup

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/endpoints/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http 18273 > /dev/null &

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: redirecting stderr to stdout


После запуска команды выше, перейдите по адресу в ngrok и подождите  пока появится DAG с именем dag

### Задачи

In [45]:
# Используя BashOperator написать команду которая будет чистить папку с логами Airflow.
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.bash import BashOperator

dag = DAG('dag',
          schedule_interval=timedelta(days=1),
          start_date=days_ago(1))


bach_op = BashOperator(
    task_id='start_task',
    bash_command="rm -r /root/airflow/logs",
    dag=dag)

bach_op

<Task(BashOperator): start_task>

In [46]:
# Используя SimpleHttpOperator обратиться по адресу
# https://www.random.org/integers/?num=1&min=1&max=5&col=1&base=2&format=plain
# и записать результат в xcom.
# { "args": {}, "headers": { "Accept": "*/*", "Accept-Encoding": "gzip, deflate", "Host": "www.httpbin.org", "User-Agent": "python-requests/2.27.1",
# "X-Amzn-Trace-Id": "Root=1-648da2af-6e86c1400b02ef3f4800085f" }, "origin": "34.125.25.165", "url": "https://www.httpbin.org/get" }

from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.providers.http.operators.http import SimpleHttpOperator

dag = DAG('dag2',
          schedule_interval=timedelta(days=1),
          start_date=days_ago(1))


task1 = SimpleHttpOperator(
    task_id="get_op",
    http_conn_id="http_default",
    method="GET",
    endpoint="get",
    dag=dag)

task1

<Task(SimpleHttpOperator): get_op>

In [47]:
# Переопределить стандартный оператор Dummy так чтобы он пушил в Xcom случайное число от 0 до 9.
# По умолчанию, результат выполнения метода execute попадает в Xcom.
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.models import BaseOperator
from jinja2 import Template


class DummyOperator(BaseOperator):
    ui_color = '#e8f7e4'
    #inherits_from_dummy_operator = True

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

    def execute(self, context):
        t = Template ('{{ range(0, 9) | random }}')
        return t.render()


dag = DAG('dag3',schedule_interval='@daily', start_date=days_ago(1))
t1 = DummyOperator(task_id='task_1', dag=dag)
t2 = DummyOperator(task_id='task_2',dag=dag)

t1 >> t2

<Task(DummyOperator): task_2>

In [48]:
# Создать руками Connection с именем custom_conn_id
# Host: google.com
# Login: user
# Password: 12345
# Сделать даг в котором вы обращаетесь к данному подключению и поочереди достаете Host, Login, Password
# После того как вы достанете вам необходимо с помощью кода записать эти 3 значения в формате json в Variables
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator
from airflow.hooks.base import BaseHook
from airflow.models import Variable

dag = DAG('dag4',schedule_interval='@daily', start_date=days_ago(1))

def conn_to_json(conn_id):
  conn = BaseHook.get_connection(conn_id)
  json = {'Password': conn.password,
         'Login': conn.login,
         'URI': conn.get_uri()
         }
  Variable.set(conn_id, json)

t1 = PythonOperator(task_id='task_1',
                    python_callable= conn_to_json,
                    op_args = ['custom_conn_id'],
                    dag=dag)

t1

<Task(PythonOperator): task_1>

In [49]:
# Используя HttpSensor обратиться по адресу
# https://www.random.org/integers/?num=1&min=1&max=5&col=1&base=10&format=plain
# Если ответ будет равен 5 то вернуть True чтобы сенсор завершился, также добавить параметр окончания действия сенсора 1 минутой
import airflow
from airflow import DAG
from airflow.providers.http.sensors.http import HttpSensor
import json

dag = DAG('dag5',schedule_interval='@daily', start_date=airflow.utils.dates.days_ago(1),)

def response_check(response, task_instance):
  if response.json() == 5:
    return True

sensor = HttpSensor(
    task_id='http_sensor',
    http_conn_id='http_default',
    endpoint='',
    response_check=response_check,
    poke_interval=10,
    timeout=60,
    dag=dag)

In [50]:
# Сгенерировать 5 DAG таким образом чтобы в каждом DAG генерировалось по 10 Task идущих параллельно. Использовать DummyOperator для задач.
# Имена DAG выбрать по такому шаблону dag_number.
from airflow import DAG
import numpy as np
from datetime import datetime, timedelta
from airflow.utils.dates import days_ago
from airflow.operators.dummy import DummyOperator

def create_dag(dag_id,
               dag_number,
               default_args,
               schedule='@daily'):

    dag = DAG(dag_id,
              schedule_interval=schedule,
              default_args=default_args)

    with dag:
      tasks_list=list()
      for i in range(0, 10):
        tasks_list.append(DummyOperator(task_id=f'task_{i}', dag=dag))
      tasks_list

    return dag


# Код для генерации дагов через range()
for dag_number in range(0, 5):
    number = ''
    if dag_number > 0:
      number = '_' + str(dag_number)

    dag_id = f'dag{number}'

    # Настройки по умолчанию
    default_args = {'owner': 'airflow',
                    'start_date': datetime(2021, 1, 1)
                    }

    # globals возвращает словарь с глобальной таблицей объектов — словарь текущего модуля.
    globals()[dag_id] = create_dag(dag_id, dag_number, default_args)


Даг нужно написать в файл /root/airflow/dags/dag.py. Проверку можно сделать в веб интерфейсе. Прежде чем даг появится, может пройти ~ 2-3 минут.